#1. Import Data

In [ ]:
import pandas as pd

In [ ]:
payment_report = pd.read_csv('/content/drive/MyDrive/DA/DAC/3. Python/payment_report.csv')
payment_report.head()

,report_month,payment_group,product_id,source_id,volume
0,2023-01,payment,12,45,624110375
1,2023-01,payment,17,45,335715113
2,2023-01,payment,18,45,737784466
3,2023-01,payment,19,45,120963069
4,2023-01,payment,20,45,319653158


In [ ]:
product = pd.read_csv('//content/drive/MyDrive/DA/DAC/3. Python/product.csv')
product.head()

,product_id,category,team_own
0,17,PXXXXXB,ASD
1,18,PXXXXXB,ASD
2,20,PXXXXXB,ASD
3,287,PXXXXXB,ASD
4,372,PXXXXXB,ASD


In [ ]:
transactions = pd.read_csv('/content/drive/MyDrive/DA/DAC/3. Python/transactions.csv')
transactions.head()

,transaction_id,merchant_id,volume,transType,transStatus,sender_id,receiver_id,extra_info,timeStamp
0,3002692434,5,100000,24,1,10199794.0,199794.0,NaN,1682932054455
1,3002692437,305,20000,2,1,14022211.0,14022211.0,NaN,1682932054912
2,3001960110,7255,48605,22,1,NaN,10530940.0,NaN,1682932055000
3,3002680710,2270,1500000,2,1,10059206.0,59206.0,NaN,1682932055622
4,3002680713,2275,90000,2,1,10004711.0,4711.0,NaN,1682932056197


#2. Merge payment_report.csv with product.csv

In [ ]:
payment_enriched=payment_report.merge(product,on='product_id',how='left')
payment_enriched.head()

,report_month,payment_group,product_id,source_id,volume,category,team_own
0,2023-01,payment,12,45,624110375,PXXXXXT,ASD
1,2023-01,payment,17,45,335715113,PXXXXXB,ASD
2,2023-01,payment,18,45,737784466,PXXXXXB,ASD
3,2023-01,payment,19,45,120963069,PXXXXXM2,ASD
4,2023-01,payment,20,45,319653158,PXXXXXB,ASD


#3. EDA
**Payment_enriched**

*1. Missing data:*
- category: 22 null values => Change to others
- team_own: 22 null values => Change to others

*2. incorrect data types*
- report_month (type: object) => Action: Change to datetime
- source_id (type: int64) => Action: Change to strings
- product_id (type: int64) => Action: Change to strings

**Transactions**

*1. Duplicate data*
- transaction_id => Action: Drop duplicate data

*2. Missing data:*
- sender_id: 49059 null values => No Action as senders could be already deleted in system
- receiver_id: 164795 null values => Action: Delete rows with receiver_id null as receiver information could be
- extra_info: 1317907 null values => No Action

*3. Incorrect data types*
- transaction_id (type: int64) => Action: Change to strings
- merchant_id (type: int64) => Action: Change to strings
- transType (type: int64) => Action: Change to strings
- transStatus (type: int64) => Action: Change to strings
- sender_id (type: float64) => Action: Change to strings
- receiver_id (type: float64) => Action: Change to strings
- timestamp (type: object) => Action: Change to datetime

##Payment_enriched

In [ ]:
#Check Shape
payment_enriched.shape

(919, 7)

In [ ]:
# Check column
payment_enriched.columns

Index(['report_month', 'payment_group', 'product_id', 'source_id', 'volume',
       'category', 'team_own'],
      dtype='object')

In [ ]:
#Check Infor
payment_enriched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   report_month   919 non-null    datetime64[ns]
 1   payment_group  919 non-null    object        
 2   product_id     919 non-null    object        
 3   source_id      919 non-null    object        
 4   volume         919 non-null    int64         
 5   category       919 non-null    object        
 6   team_own       919 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 50.4+ KB


In [ ]:
#Count null value
payment_enriched.isnull().sum()

,0
report_month,0
payment_group,0
product_id,0
source_id,0
volume,0
category,0
team_own,0


In [ ]:
#Descriptive Statistics
payment_enriched.describe()

,report_month,volume
count,919,9.190000e+02
mean,2023-02-19 06:05:05.549510400,1.978574e+08
min,2023-01-01 00:00:00,5.500000e+03
25%,2023-02-01 00:00:00,1.250000e+06
50%,2023-03-01 00:00:00,7.982015e+06
75%,2023-04-01 00:00:00,5.447599e+07
max,2023-04-01 00:00:00,1.383171e+10
std,NaN,8.367595e+08


In [ ]:
#Counting Unique value
payment_enriched.nunique()

,0
report_month,4
payment_group,2
product_id,311
source_id,4
volume,896
category,16
team_own,4


In [ ]:
#Replace null value in category & team_own by Others value
payment_enriched[payment_enriched["category"].isnull() | payment_enriched["team_own"].isnull()]
payment_enriched["category"].fillna("Others", inplace = True)
payment_enriched["team_own"].fillna("Others", inplace = True)
payment_enriched

<ipython-input-129-ad62b7f076e4>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  payment_enriched["category"].fillna("Others", inplace = True)
<ipython-input-129-ad62b7f076e4>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=T

,report_month,payment_group,product_id,source_id,volume,category,team_own
0,2023-01-01,payment,12,45,624110375,PXXXXXT,ASD
1,2023-01-01,payment,17,45,335715113,PXXXXXB,ASD
2,2023-01-01,payment,18,45,737784466,PXXXXXB,ASD
3,2023-01-01,payment,19,45,120963069,PXXXXXM2,ASD
4,2023-01-01,payment,20,45,319653158,PXXXXXB,ASD
...,...,...,...,...,...,...,...
914,2023-04-01,payment,15067,45,1504000,PXXXXXR,ASL
915,2023-04-01,refund,1976,37,3542271587,Others,Others
916,2023-04-01,refund,1976,38,13831708189,Others,Others
917,2023-04-01,refund,1976,39,1905435543,Others,Others


In [ ]:
# Fix incorrect data types
payment_enriched["report_month"]=pd.to_datetime(payment_enriched["report_month"])
payment_enriched["source_id"]=payment_enriched["source_id"].astype(str)
payment_enriched["product_id"]=payment_enriched["product_id"].astype(str)
payment_enriched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   report_month   919 non-null    datetime64[ns]
 1   payment_group  919 non-null    object        
 2   product_id     919 non-null    object        
 3   source_id      919 non-null    object        
 4   volume         919 non-null    int64         
 5   category       919 non-null    object        
 6   team_own       919 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 50.4+ KB


##Transactions

In [ ]:
#Check Shape
transactions.shape

(1324002, 9)

In [ ]:
transactions.columns

Index(['transaction_id', 'merchant_id', 'volume', 'transType', 'transStatus',
       'sender_id', 'receiver_id', 'extra_info', 'timeStamp'],
      dtype='object')

In [ ]:
#Check Infor
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324002 entries, 0 to 1324001
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   transaction_id  1324002 non-null  int64  
 1   merchant_id     1324002 non-null  int64  
 2   volume          1324002 non-null  int64  
 3   transType       1324002 non-null  int64  
 4   transStatus     1324002 non-null  int64  
 5   sender_id       1274943 non-null  float64
 6   receiver_id     1159207 non-null  float64
 7   extra_info      6095 non-null     object 
 8   timeStamp       1324002 non-null  int64  
dtypes: float64(2), int64(6), object(1)
memory usage: 90.9+ MB


In [ ]:
#Count null value
transactions.isnull().sum()

,0
transaction_id,0
merchant_id,0
volume,0
transType,0
transStatus,0
sender_id,49059
receiver_id,164795
extra_info,1317907
timeStamp,0


In [ ]:
#Descriptive Statistics
transactions.describe()

,transaction_id,merchant_id,volume,transType,transStatus,sender_id,receiver_id,timeStamp
count,1.324002e+06,1.324002e+06,1.324002e+06,1.324002e+06,1.324002e+06,1.274943e+06,1.159207e+06,1.324002e+06
mean,3.002233e+09,2.460318e+03,2.388059e+05,6.979222e+00,-1.204625e+01,1.033938e+08,2.084884e+08,1.683130e+12
std,1.042606e+07,3.304277e+03,9.681009e+05,7.459714e+00,5.577823e+01,6.234305e+08,9.287794e+08,1.707815e+08
min,3.000000e+09,5.000000e+00,1.000000e+00,2.000000e+00,-1.333000e+03,1.000001e+07,-6.300000e+01,1.682874e+12
25%,3.001121e+09,3.050000e+02,1.000000e+04,2.000000e+00,1.000000e+00,1.005657e+07,1.526700e+05,1.682994e+12
50%,3.002200e+09,2.250000e+03,3.000000e+04,2.000000e+00,1.000000e+00,1.094193e+07,7.025130e+06,1.683097e+12
75%,3.003255e+09,2.270000e+03,1.000000e+05,8.000000e+00,1.000000e+00,2.101266e+07,3.151238e+07,1.683269e+12
max,6.000066e+09,1.625250e+05,7.869148e+07,3.000000e+01,1.000000e+00,6.993439e+09,2.100000e+10,1.683479e+12


In [ ]:
#Counting Unique value
transactions.nunique()

,0
transaction_id,1323974
merchant_id,646
volume,49488
transType,7
transStatus,35
sender_id,173557
receiver_id,200291
extra_info,2037
timeStamp,1316590


In [ ]:
#Check & drop transactions ID duplicate value
transactions[transactions.duplicated('transaction_id',keep=False)].sort_values(by='transaction_id',ascending=True)
transactions.drop_duplicates(subset='transaction_id',keep='first',inplace=True)

In [ ]:
# Change incorrect data types to proper data types
transactions["transaction_id"]=transactions["transaction_id"].astype(str)
transactions["merchant_id"]=transactions["merchant_id"].astype(str)
transactions["transType"]=transactions["transType"].astype(str)
transactions["transStatus"]=transactions["transStatus"].astype(str)
transactions["sender_id"]=transactions["sender_id"].astype(str)
transactions["receiver_id"]=transactions["receiver_id"].astype(str)
transactions["timeStamp"]=pd.to_datetime(transactions["timeStamp"])
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1323974 entries, 0 to 1324001
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   transaction_id  1323974 non-null  object        
 1   merchant_id     1323974 non-null  object        
 2   volume          1323974 non-null  int64         
 3   transType       1323974 non-null  object        
 4   transStatus     1323974 non-null  object        
 5   sender_id       1323974 non-null  object        
 6   receiver_id     1323974 non-null  object        
 7   extra_info      6095 non-null     object        
 8   timeStamp       1323974 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 101.0+ MB


In [ ]:
#Check & drop transactions ID duplicate value
transactions[transactions.duplicated('transaction_id',keep=False)].sort_values(by='transaction_id',ascending=True)
transactions.drop_duplicates(subset='transaction_id',keep='first',inplace=True)

In [ ]:
transactions.set_index('transaction_id',inplace=True)

#Data Wrangling

**1. Top 3 product_ids with the highest volume.**

In [ ]:
print(payment_enriched.groupby("product_id")["volume"].sum().sort_values(ascending=False).head(3))

product_id
1976    61797583647
429     14667676567
372     13713658515
Name: volume, dtype: int64


**2. Given that 1 product_id is only owed by 1 team, are there any abnormal products against this rule?**

Answer: No

In [ ]:
abnormal_products=payment_enriched.groupby("product_id")["team_own"].nunique()
# payment_enriched[payment_enriched["product_id"].isin(["10033","1976","3"])]
abnormal_products[abnormal_products>1]

,team_own
product_id,


**3. Find the team has had the lowest performance (lowest volume) since Q2.2023. Find the category that contributes the least to that team.**

- Lowest performance team is APS
- lowest_performance_team_category is 25232438



In [ ]:
comparison_date = pd.Timestamp("2023-04-01")
lowest_performance_team=payment_enriched[payment_enriched["report_month"]>=comparison_date].groupby("team_own")["volume"].sum().sort_values(ascending=True).head(1)
lowest_performance_team_category=payment_enriched[(payment_enriched["report_month"]>=comparison_date) & (payment_enriched["team_own"]=="APS")].groupby("category")["volume"].sum().sort_values(ascending=True).head(1)
print(lowest_performance_team)
print(lowest_performance_team_category)

team_own
APS    51141753
Name: volume, dtype: int64
category
PXXXXXE    25232438
Name: volume, dtype: int64


**4. Find the contribution of source_ids of refund transactions (payment_group = ‘refund’), what is the source_id with the highest contribution?**

Source_id with the highest contribution is 38



In [ ]:
refund_transactions=payment_enriched[payment_enriched["payment_group"]=="refund"]
refund_transactions.groupby("source_id")["volume"].sum().sort_values(ascending=False).head(1)

,volume
source_id,
38,36527454759


**5. Define type of transactions (‘transaction_type’) for each row, given:**
- transType = 2 & merchant_id = 1205: Bank Transfer Transaction
- transType = 2 & merchant_id = 2260: Withdraw Money Transaction
- transType = 2 & merchant_id = 2270: Top Up Money Transaction
- transType = 2 & others merchant_id: Payment Transaction
- transType = 8, merchant_id = 2250: Transfer Money Transaction
- transType = 8 & others merchant_id: Split Bill Transaction
- Remained cases are invalid transactions

In [ ]:
def transaction_type(Trans_Type,Merchant_Id):
    if (Trans_Type == "2") & (Merchant_Id == "1205"):
        return "Bank Transfer Transaction"
    elif (Trans_Type == "2") & (Merchant_Id == "2260"):
        return "Withdraw Money Transaction"
    elif (Trans_Type == "2") & (Merchant_Id == "2270"):
        return "Top Up Money Transaction"
    elif (Trans_Type == "2"):
        return "Payment Transaction"
    elif (Trans_Type == "8") & (Merchant_Id == "2250"):
        return "Transfer Money Transaction"
    elif (Trans_Type == "8"):
        return "Split Bill Transaction"
    else:
        return "Invalid Transactions"
transactions["transaction_type"]=transactions.apply(lambda x: transaction_type(x['transType'], x['merchant_id']), axis=1)
transactions

,merchant_id,volume,transType,transStatus,sender_id,receiver_id,extra_info,timeStamp,transaction_type
transaction_id,,,,,,,,,
3002692434,5,100000,24,1,10199794.0,199794.0,NaN,1970-01-01 00:28:02.932054455,Invalid Transactions
3002692437,305,20000,2,1,14022211.0,14022211.0,NaN,1970-01-01 00:28:02.932054912,Payment Transaction
3001960110,7255,48605,22,1,nan,10530940.0,NaN,1970-01-01 00:28:02.932055000,Invalid Transactions
3002680710,2270,1500000,2,1,10059206.0,59206.0,NaN,1970-01-01 00:28:02.932055622,Top Up Money Transaction
3002680713,2275,90000,2,1,10004711.0,4711.0,NaN,1970-01-01 00:28:02.932056197,Payment Transaction
...,...,...,...,...,...,...,...,...,...
3003723030,305,20000,2,1,24524311.0,nan,NaN,1970-01-01 00:28:03.035672634,Payment Transaction
3003723033,2270,100000,2,1,10277242.0,277242.0,NaN,1970-01-01 00:28:03.035672876,Top Up Money Transaction
3003723036,2270,100000,2,1,10144599.0,144599.0,NaN,1970-01-01 00:28:03.035672892,Top Up Money Transaction


**6. Of each transaction type (excluding invalid transactions): find the number of transactions, volume, senders and receivers**

In [ ]:
Pivot_transactions=transactions[transactions["transaction_type"]!="Invalid Transactions"].pivot_table(index="transaction_type",aggfunc={"merchant_id":"count","volume":"sum","sender_id":"nunique","receiver_id":"nunique"})
Pivot_transactions=Pivot_transactions.rename(columns={
    "merchant_id": "Count of Transactions",
    "volume": "Total volume",
    "sender_id": "Count of Senders",
    "receiver_id": "Count of Receivers"})
Pivot_transactions

,Count of Transactions,Count of Receivers,Count of Senders,Total volume
transaction_type,,,,
Bank Transfer Transaction,14004,9271,9271,10061351762
Payment Transaction,260335,113298,102995,34385618147
Split Bill Transaction,1376,572,1323,4901464
Top Up Money Transaction,290498,110409,110409,108605618829
Transfer Money Transaction,341173,34585,39021,37032880492
Withdraw Money Transaction,33725,24814,24814,23418181420
